In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook
tqdm_notebook().pandas()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/shopee-code-league-20/_DA_Order_Brushing/order_brush_order.csv")
df.head()

In [ ]:
df['event_time'] = pd.to_datetime(df['event_time'])

In [ ]:
possible_shop = df.groupby('shopid').size() > 3

In [ ]:
def count_concentrate(row):
    shop_id = row['shopid']
    if possible_shop[shop_id] == True:
        return 0
    
    event_time = row['event_time']
    end_time = event_time + pd.Timedelta('1h')
    
    # 看這筆訂單後一小時有沒有超過3筆訂單 有才計算
    orders = df[(df['shopid']==shop_id) & (df['event_time'] >= event_time) & (df['event_time'] <= end_time)]
    order_num = len(orders)
    if order_num < 3:
        return 0
    
    else:
        buyer_num = len(orders['userid'].unique())
    con = order_num / buyer_num
    return con

In [ ]:
df['con_rate'] = df.progress_apply(count_concentrate, axis=1)

In [ ]:
brush_shop = df[df['con_rate'] >= 3].sort_values('shopid').set_index(['shopid'])

In [ ]:
brush_shop

In [ ]:
df[df['shopid']==3124091].groupby('userid').size() == 3

In [ ]:
result = pd.DataFrame()
result['shopid'] = df.shopid.unique()

In [ ]:
def get_brush_id(row):
    shop_id = row['shopid']
    if shop_id not in brush_shop.index:
        return 0
    
    dff = df[df['shopid']==shop_id]
    dff_size = dff.groupby('userid').size()
    max_num = dff_size.max()
    user_id = sorted(dff_size[dff_size==max_num].index.tolist())
    user_id = [str(i) for i in user_id]
    return '&'.join(user_id)

In [ ]:
result['userid'] = result.progress_apply(get_brush_id, axis=1)

In [ ]:
result

In [ ]:
result.to_csv('submission.csv', index=None)